# IA learns logical Operator AND

This is an example of an Artificial Intelligence learning the AND operator.
We are using a multiperceptron.

- A perceptron is a NN (Neural Network) were you have the entries and one layer which is the output one.
- On a multiperceptron, we have hidden layers appart from the output ine and the entries.
- Also, is a supervised model where we are going to give the model what output should it take, and applaying backpropagation, calculating it's gradient decent and updating the hyperparameters.

    NN with hidden layer   |           Neuron         
:-------------------------:|:-------------------------:
![](images/NN_hidden.png)  | ![](images/neuron.png)


#### 1º Data set
| e1| e2| s |
|---|---|---|
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

#### 2º Activation function

Sigmoidal in each neuron, we could try some others, like linear, ReLU (Rectified Linear Unit) or tanh.

#### 3º Model

This model is going to have just one hidden layer, output layer and 2 entries (e1, e2).
Is a supervised model.

In [ ]:
# imports
import numpy as np

# constants
LEARNING_RATE = 1.5
N_LAYERS = 2 # 1 hidden layer and 1 output layer, the entry one does not count as a layer
N_NEURONS_HIDDEN_LAYER = 3
N_NEURONS_ENTRIES = 2
N_NEURONS_OUTPUT = 1

# data sets

##### train case #####

# train_set_X -> 2 inputs and 4 train examples
train_set_X = np.array([
    # np.reshape([0, 0], (1, 2)),
    np.reshape([0, 1], (1, 2)),
    # np.reshape([1, 0], (1, 2)),
    np.reshape([1, 1], (1, 2)),
])
print(train_set_X[0].shape)

train_set_Y = np.reshape([
    # 0,
    0,
    # 0,
    1,
], (2, 1))


##### test case #####

test_set_X = np.array([
    np.reshape([0, 0], (1, 2)),
    np.reshape([0, 1], (1, 2)),
    np.reshape([1, 1], (1, 2)),
])

test_set_Y = np.reshape([
    0,
    0,
    1,
], (1, 3))

##### hyperparameters (W, b) #####
# bias from entries is 0
# b0 = 0

# first layer weights
# [first entry, second entry]
w1 = np.array([
    [np.random.uniform(0, 0.5), np.random.uniform(0.3, 0.6), np.random.uniform(-1, -0.5)], # from e1 to n0, n1, n2
    [np.random.uniform(0, 0.5), np.random.uniform(0.3, 0.6), np.random.uniform(-1, -0.5)], # from e2 to n0, n1, n2
])

b1 = np.reshape([
    np.random.uniform(-0.2, 0), # np.random.uniform(0, 1),
    np.random.uniform(-1, -0.5),
    np.random.uniform(0, 1),
], (1, 3))

print(b1)

w2 = np.reshape([
    np.random.uniform(-1, -0.5), # np.random.uniform(0, 1), # first neuron from hidden layer
    np.random.uniform(-0.5, 0), # second neuron from hidden layer
    np.random.uniform(0.5, 1), # third neuron from hidden layer
], (3, 1))

b2 = np.reshape(np.random.uniform(0, 0.5), (1, 1)) # np.random.uniform(0, 1)

### Sigmoid function and its derivative

- Sigmoid: $f(x) = \frac{1}{1+\mathrm{e}^{-x}}$
- Sigmoid Derivate: $f'(x) = f(x)*(1-f(x))$

The derivate is use to calculate how much does our model need to "rectificate", later you will see implement a function $\delta$ that implements it.

<img
     src="https://cdn-images-1.medium.com/max/1600/0*rBCtCuf6-L5kYHGE.png"
     style="width:350px;height:150px;"
/>

In [ ]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

### Forward propagation

Here we implement the forward propagation. It is the way to calculate the output, so in this step we have to calculate the activation function. (in this case)
1. Calculate the linear function with W (Weight), b (bias) and e (entries) in the first layer (which is the hidden layer)
2. Apply activation function (sigmoid) with the last output
3. Calculate the linear function of the last layer (output layer) as in the first step, but with a little difference, here we need to apply W (from the last layer) instead of e (entries)
4. Apply activation function

<img src="images/NN_forward_prop.png" style="width:500px;height:300px;" />

In [ ]:
def forward_propagation(X, w, b):
    # weigth from bias is 1
    # calculate f(x) and a(x), we can do this by:
    # try to uncomment this to see shapes print(X.shape, w.T.shape, b.shape)
    return sigmoid(np.matmul(X, w)+b)

###### Aclaration

When we have layers, we want to get the size of the **Neurons** you have in the actual layer with the **Neurons** from the next layer. Example of the NN in the img.

<img src="images/forward_prop_steps.png" style="width:550px;height:350px;" />

In [ ]:
# An example for the forwarding for 2 layers.

# We get an array with dimensions (1, 3)
activation_func1 = forward_propagation(train_set_X[0], w1, b1)
print(activation_func1.shape, activation_func1.squeeze())
# We get an array with dimensions (1, 1)
activation_func2 = forward_propagation(activation_func1, w2, b2)
print(activation_func2.shape, activation_func2.squeeze())

### Backward propagation

In this step we have the output **s** and the desire output **d**, to calculate how much the hyperparameters (w, b) has to change to get the output we want, we do the next step, called _Backward propagation_, here do the next:

1. Calculate $\delta^{(k)} = (d - s^{(k)}) * s^{(k)} * (1-s^{(k)})$ from last layer. Here we calculate the error.
2. Calculate $\Delta w$ for the last layer ($\Delta w = \alpha * s^{(k-1).T} * \delta^{(k)}$) it gives us how we need to change the weights to be more accurate for the desire output. (last layer)
3. Calculate $\Delta b$ for the last layer ($\Delta b = \alpha * \delta^{(k)}$)
4. Update weights $w^{(k)}(t+1) = w^{(k)}(t) + \delta w^{(k)}(t+1)$
5. Update bias $b^{(k)}(t+1) = b^{(k)}(t) + \delta b^{(k)}(t+1)$
Now the same with the hidden layer, but in a different way, because we don't have a desire output
6. Calculate $V^{(k)} = \delta^{(k+1)} * w^{(k+1)}$ the error in the last layer.
7. Calculate $\delta^{(k)} = V^{(k)} * s^{k} * (1-s^{(k)})$ the error each neuron of current layer.
8. Calculate $\Delta w$ for the last layer ($\Delta w = \alpha * s^{(k-1).T} * \delta^{(k)}$) how much we change the neurons weights on the hidden layers.
9. Calculate $\Delta b$ for the last layer ($\Delta b = \alpha * \delta^{(k)}$)
10. Update weights $w^{(k)}(t+1) = w^{(k)}(t) + \delta w^{(k)}(t+1)$
11. Update bias $b^{(k)}(t+1) = b^{(k)}(t) + \delta b^{(k)}(t+1)$

from 6 to 7 has to be repeted for each hidden layer.

- k -> layer
- s -> activation function output
- d -> desire output
- t -> number of times the model has learned
- $\alpha$ -> learning_rate
- $\delta$ -> lowercase delta
- $\Delta$ -> uppercase delta

remmember $s^0$ are the entries.

<img src="images/NN_backward_prop.png" style="width:550px;height:450px;" />

In [ ]:
# I know sounds like a lot, but wish with the img you can see it better
# now what we need to do is the function back_prop, here we generalize all from above
def back_prop(s, w=None, Y=None, last_err=None, learning_rate=LEARNING_RATE):
    # Y -- desire output, in hidden layers, we pass the activation function from the last layer
    # s -- s[0] <- current activation function
    #      s[1] <- last activation function (last because is the layer which is back to the current one)
    
    # remember, last activation function, since there is only one neuron, we have no matrix
    # other wise we need to do a matrix multiplication.
    err = 0
    # this means we are on the last layer
    if Y != None:
        # lower delta
        err = np.dot(np.dot((Y-s[0]), s[0]), (1-s[0]))
    else:
        v = np.dot(last_err, w.T)
        err = v * s[0] * (1-s[0])

    del_w = learning_rate * np.dot(s[1].T, err)
    del_b = learning_rate * err
    
    hyperparams = {
        'err': err,
        'del_w': del_w,
        'del_b': del_b,
    }
    
    return hyperparams

In [ ]:
# example with the forward prop in the last cells
# we had e1=e2=0, and the deire output is 0

print('---------------- output layer ----------------\n')

# last layer
s = [activation_func2, activation_func1]
err2 = back_prop(s, Y=0)
print(f"error: {err2['err'].shape} \n", err2['err'])
print(f"\ndel_w: {err2['del_w'].shape} \n", err2['del_w'])
print(f"\ndel_b: {err2['del_b'].shape} \n", err2['del_b'])

print('---------------- hidden layer ----------------\n')

# hidden layer
# this part should be a loop so that we cover every hidden layer with back prop, but since it is an example and we 
# just have one layer, we are going to omit it.
s = [activation_func1, train_set_X[0]]
err1 = back_prop(s, w=w2, last_err=err2['err'])
print(f"error: {err1['err'].shape} \n", err1['err'])
print(f"\ndel_w: {err1['del_w'].shape} \n", err1['del_w'])
print(f"\ndel_b: {err1['del_b'].shape} \n", err1['del_b'])

In [ ]:
# update hyperparameters
def update(w, del_w, b, del_b):
    return  w + del_w, b + del_b

In [ ]:
history_hyper = [[{"w2": w2, "b2": b2}],[{"w1": w1, "b1": b1}]]

print(f'Old hyperparams\n- w2: {w2}\n- b2: {b2}\n- w1: {w1}\n- b1: {b1}')

# update second layer
w2_ex, b2_ex = update(err2['del_w'], w2, err2['del_b'], b2)
history_hyper[0].append({"w2": w2_ex, "b2": b2_ex})
w1_ex, b1_ex = update(err1['del_w'], w1, err1['del_b'], b1)
history_hyper[1].append({"w1": w1_ex, "b1": b1_ex})

print(f'\nNew hyperparams\n- w2: {w2_ex}\n- b2: {b2_ex}\n- w1: {w1_ex}\n- b1: {b1_ex}')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# this is done just for this model, but it can be generalize
def print_l(history_hyper, iteration, n_total_layers=N_LAYERS, n_neurons_hidden_l=[N_NEURONS_HIDDEN_LAYER]):
    neuron_layers_W = [[] for _ in range(len(history_hyper))]
    neuron_layers_b = [[] for _ in range(len(history_hyper))]

    # print(history_hyper)
    
    # output layer
    # ---------------------------------- #
    output_layer = history_hyper[0]
    print("\033[91m", "\033[1m", "n_total_layers: ", n_total_layers, "\033[0m")
    # print(history_hyper)
    
    for i in range(len(output_layer[0][f"w{len(history_hyper)}"].squeeze())):
        neuron_layers_W[n_total_layers-1].append([])
    for i in range(len(output_layer[0][f"b{len(history_hyper)}"].tolist())):
        neuron_layers_b[n_total_layers-1].append([])

    # process data
    for neurons in output_layer:
        for i in range(len(neurons[f"w{n_total_layers}"].squeeze())):
            neuron_layers_W[n_total_layers-1][i].append(neurons[f"w{n_total_layers}"].squeeze()[i])
            # print(f"{i} - W{n_total_layers}:", neurons[f"w{n_total_layers}"].squeeze()[i])

        for i in range(len(neurons[f"b{n_total_layers}"].tolist())):
            neuron_layers_b[n_total_layers-1][i].append(neurons[f"b{n_total_layers}"].tolist()[i][0])
            # print(f"b{n_total_layers}:", neurons[f"b{n_total_layers}"].tolist()[i][0])
    
    # hidden layer
    # ---------------------------------- #
    hidden_layer = history_hyper[1:]
    
    # print("hidden_layer: ", hidden_layer)
    
    n = n_total_layers-1
    for num_hidden_l in range(len(hidden_layer)):
        # same number of neurons, for w and b
        for _ in range(len(hidden_layer[num_hidden_l][0][f"b{n}"].squeeze())):
            neuron_layers_b[n-1].append([])
        n -= 1

    n = n_total_layers-1
    for layers in range(len(hidden_layer)):
        if n == 1:
            for _ in range(len(hidden_layer[layers][0][f"w{n}"])*len(hidden_layer[layers][0][f"w{n}"][0])):
                neuron_layers_W[n-1].append([])
        else:
            for _ in range(len(hidden_layer[layers][0][f"w{n}"])*len(hidden_layer[layers+1][0][f"w{n-1}"])):
                neuron_layers_W[n-1].append([])
        n -= 1
    
    n = n_total_layers-2
    # process hidden layers hyperparams data
    for layer in hidden_layer:
        for t in layer:
            number = 0 # the position for each weight
            for allW in t[f"w{n+1}"]:
                for w in allW:
                    neuron_layers_W[n][number].append(w)
                    number+=1
            
            number = 0 # the position for each bias
            for allB in t[f"b{n+1}"]:
                for b in allB:
                    neuron_layers_b[n][number].append(b)
                    number+=1
        n-=1

    # ---------------------------------- #

    # for each hidden layer we have
    for i in range(len(neuron_layers_W)-1):
        # create subplots for w and b
        hidden_layer_fig_w, hidden_layer_ax_w = plt.subplots(1, 1, figsize=(10, 10), sharey=False)
        for j in range(len(neuron_layers_W[i])):
            hidden_layer_ax_w.plot(neuron_layers_W[i][j], label=f"W neuron {(j%n_neurons_hidden_l[i])+1}")
            hidden_layer_ax_w.legend(loc="upper right")
            hidden_layer_ax_w.set_xlim(0, iteration)
        hidden_layer_fig_w.suptitle(f"Weights from layer {i+1} to {i+2}, neurons from 0 to {(j%n_neurons_hidden_l[i])+1}")

        hidden_layer_fig_bias, hidden_layer_ax_bias = plt.subplots(1, 1, figsize=(10, 10), sharey=False)
        for j in range(len(neuron_layers_b[i])):
            hidden_layer_ax_bias.plot(neuron_layers_b[i][j], label=f"bias from neuron {j+1}")
            hidden_layer_ax_bias.legend(loc="upper right")
            hidden_layer_ax_bias.set_xlim(0, iteration)
        hidden_layer_fig_bias.suptitle(f"Bias from layer {i+1}, neurons from 0 to {(j%n_neurons_hidden_l[i])+1}")


    output_layer_fig, output_layer_ax = plt.subplots(1, len(neuron_layers_W[n_total_layers-1]), figsize=(15, len(neuron_layers_W[n_total_layers-1])), sharey=False)
    for i in range(len(neuron_layers_W[n_total_layers-1])):  
        output_layer_ax[i].plot(neuron_layers_W[n_total_layers-1][i], label=f"W neuron {i}")
        output_layer_ax[i].legend(loc="upper right")
        output_layer_ax[i].set_xlim(0, iteration)
    output_layer_fig.suptitle('Weights from output')

    _, output_layer_ax_bias = plt.subplots(figsize=(4.3, 3), sharey=False)
    output_layer_ax_bias.plot(neuron_layers_b[n_total_layers-1][0], label="bias from output neuron")
    output_layer_ax_bias.set_xlim(0, iteration)
    output_layer_ax_bias.legend(loc="upper right")
        
    plt.show()

In [ ]:
# Just for you to see how graphicaly the weights and bias has changed. [the example]
print_l(history_hyper, 1, n_total_layers=2)

## Grouping forward and backward making the model

At this point we have implemented:
- Activation function ($\sigma$)
- Forward propagation
- Backward propagation

What the model has to do:
1. Train
    1. **Forward propagation** with a data set
    2. **Backward propagation**, learn from its result
    3. **Update** hyperparameters
2. Test
    1. **Predict**, this means, given a situation, with what the NN has learned give its output.
    2. **Accuracy**, some times we want to know how well our NN is doing, we do this by comparing the outputs with desired outputs, depending on how many outputs has correct, its accuracy is better or worst.

Step one (Train) is done X time, those X times are the number, of epochs (when you train 1 time the NN with the whole data set) you train the whole data set.
    
_Note: A, B, C should be 1, 2, 3_

In [ ]:
def model(X, w, b, Y, epochs=10, print_progress=False, plt_all_progress=True, learning_rate=LEARNING_RATE, n_layers=N_LAYERS, n_neurons_hidden_l=[N_NEURONS_HIDDEN_LAYER]):
    # X - training data set (entries)
    # w - weights
    #     + w[0] - first layer weights
    #     + w[n] - next n layer weights
    # b - bias from training data set
    #     + b[0] - first layer bias
    #     + b[n] - next n layer bias
    # Y - desire output data set
    # epochs - number of repetitions
    # print_progress - print all the progress from each data set
    # plot_progress - plot in a graph the progress

    # Some history if you need it 
    activation_func_history = [[] for _ in range(len(X))]
    err_history = [[] for _ in range(len(X))]
    
    history_hyper = [[{"w"+str(i): w[i-1], "b"+str(i): b[i-1]}] for i in range(len(w), 0, -1)]

    # Training
    for i in range(epochs):
        if print_progress:
            print(f"----------------------- epoch {i+1} -----------------------")
        for j in range(len(X)): # j gives us the training set we are using

            # ---------- Forward propagation with each layer ---------- #
            s = []
            # first from entry to first hidden layer
            s.append(forward_propagation(X[j], w[0], b[0]))
            # from each hidden layer that exists to the output layer
            for layer in range(n_layers-1):
                s.append(forward_propagation(s[layer], w[layer+1], b[layer+1]))

            activation_func_history[j].append(s)
            
            # now we all outputs from each neuron -> s
            if print_progress:
                print(s)
                print(f"------------- activation_func_history [{j}] -------------")
                print(activation_func_history[j])
                print("----------------------------------------------------------")

            # ---------- Backward propagation from output to entries ---------- #
            # first we need to calculate the error from the desire output
            # remember, here we have the output layer in the first position, not as in the forward prop (s)
            
            # so we have that err[0] is the error from the output layer and err[n_layers-1] is the error
            # from the last hidden layer
            
            err = []
            err.append(back_prop([s[n_layers-1], s[n_layers-2]], Y=Y[j], learning_rate=learning_rate))
            # next step is to calculate the error each neuron has done
            for layer in range(n_layers-2, 0, -1): # from n_layers-2 to 0
                # n_layers-2 because we did the first step (-1) before, now the next one (-1)
                err.append(back_prop([s[layer], s[layer+1]], w=w[layer+1], last_err=err[layer-1]['err'], learning_rate=learning_rate))

            # last, and more important, calculate the error from weights of entry layer
            err.append(back_prop([s[0], X[j]], w=w[1], last_err=err[n_layers-2]['err']))

            err_history[j].append(err)

            if print_progress:
                print(err)
                print(f"------------- activation_func_history [{j}] -------------")
                print(err_history[j])
                print("----------------------------------------------------------")

            # ---------- Update values ---------- #
            # remmember, when we store the error of each layer, we store it from output layer to the first one
            # and w, b is the other way
            for i in range(n_layers):
                weight, bias = update(w[i], err[n_layers-i-1]['del_w'], b[i], err[n_layers-i-1]['del_b'])
                w[i] = weight
                b[i] = bias
                history_hyper[n_layers-i-1].append({f"w{i+1}": weight, f"b{i+1}": bias})
    
    if plt_all_progress:
        print_l(history_hyper, epochs, n_total_layers=n_layers, n_neurons_hidden_l=n_neurons_hidden_l)
    
    w_b = []
    #print(len(history_hyper[0]))
    #print(history_hyper[0][len(history_hyper[0])-1])
    for i in range(n_layers):
        # print(f"w{n_layers-i}", history_hyper[i][len(history_hyper[i])-1][f"w{n_layers-i}"])
        w_b.append({
            f"w{n_layers-i}": history_hyper[i][len(history_hyper[i])-1][f"w{n_layers-i}"],
            f"b{n_layers-i}": history_hyper[i][len(history_hyper[i])-1][f"b{n_layers-i}"],
        })

    return w_b

In [ ]:
w = [w1, w2]
b = [b1, b2]
# N_LAYERS = 2
print(w)
print(b)
# print("N_LAYERS:", N_LAYERS)
hyper_params_after = model(train_set_X, w, b, train_set_Y, epochs=200, n_layers=N_LAYERS,
                                   plt_all_progress=True, learning_rate=0.05)

print("-------------------------------------------------------------------")
w_next = [[], []]
b_next = [[], []]
for i in range(N_LAYERS):
    w_next[N_LAYERS-i-1] = hyper_params_after[i][f"w{N_LAYERS-i}"]
    b_next[N_LAYERS-i-1] = hyper_params_after[i][f"b{N_LAYERS-i}"]

print(w_next)
print(b_next)

In [ ]:
# test_set_X
# test_set_Y
def predict(X, w, b, n_layers=N_LAYERS):
    s = []
    # first from entry to first hidden layer
    s.append(forward_propagation(X, w[0], b[0]))
    # from each hidden layer that exists to the output layer
    for layer in range(n_layers-1):
        s.append(forward_propagation(s[layer], w[layer+1], b[layer+1]))

    print(s[n_layers-1])

    return (1 if s[n_layers-1] > 0.5 else 0)

In [ ]:
def accuracy(outputs, desire_outputs):
    correct = 0
    for i in range(len(outputs)):
        if outputs[i] == desire_outputs.squeeze()[i]:
            correct+=1

    return (correct/len(outputs))*100

In [ ]:
predictions = [predict(test_set_X[0], w_next, b_next, n_layers=2), predict(test_set_X[1], w_next, b_next, n_layers=2),
               predict(test_set_X[2], w_next, b_next, n_layers=2)]

print(f'{test_set_X[0].squeeze()[0]} AND {test_set_X[0].squeeze()[1]} = {predictions[0]}') # 0
print(f'{test_set_X[1].squeeze()[0]} AND {test_set_X[1].squeeze()[1]} = {predictions[1]}') # 0
print(f'{test_set_X[2].squeeze()[0]} AND {test_set_X[2].squeeze()[1]} = {predictions[2]}') # 1

acc = accuracy(predictions, test_set_Y)

print(f"Accuracy: {acc}% from {len(predictions)} tests")

# This is all!!!

Hope you understand all.

Also, you can use this model with the OR logic, and try with XOR!!!

Don't forget to change values!! and play with it!!
Try adding a tanh as activation function, or another one!

Give me a Star , that helps me to continue doing these things.

Have a good day, and see you!!

Made by: **@blitty-codes**

Just one more thing, for you to notice how important the data set you are training with is the biggest thing. Try combinations like:


------------------------------
```
train_set_X = np.array([
    # np.reshape([0, 0], (1, 2)),
    # np.reshape([0, 1], (1, 2)),
    np.reshape([1, 0], (1, 2)),
    np.reshape([1, 1], (1, 2)),
])
print(train_set_X[0].shape)

train_set_Y = np.reshape([
    # 0,
    # 0,
    0,
    1,
], (3, 1))

------------------------------
train_set_X = np.array([
    # np.reshape([0, 0], (1, 2)),
    np.reshape([0, 1], (1, 2)),
    np.reshape([1, 0], (1, 2)),
    # np.reshape([1, 1], (1, 2)),
])
print(train_set_X[0].shape)

train_set_Y = np.reshape([
    # 0,
    0,
    0,
    # 1,
], (3, 1))

------------------------------
train_set_X = np.array([
    np.reshape([0, 0], (1, 2)),
    # np.reshape([0, 1], (1, 2)),
    np.reshape([1, 0], (1, 2)),
    np.reshape([1, 1], (1, 2)),
])
print(train_set_X[0].shape)

train_set_Y = np.reshape([
    0,
    # 0,
    0,
    1,
], (3, 1))

------------------------------
train_set_X = np.array([
    # np.reshape([0, 0], (1, 2)),
    np.reshape([0, 1], (1, 2)),
    np.reshape([1, 0], (1, 2)),
    np.reshape([1, 1], (1, 2)),
])
print(train_set_X[0].shape)

train_set_Y = np.reshape([
    # 0,
    0,
    0,
    1,
], (3, 1))
```
------------------------------
and run all cells, you'll see how the cuallity of the data and the cuantity matters.

In [ ]:
# XOR

# Try to do XOR with the minimum neurons possible, so here we will need to use more than 3 neurons with 2 test examples.

# constants
LEARNING_RATE = 1.5
N_LAYERS = 3 # 1 hidden layer and 2 output layer, the entry one does not count as a layer
N_NEURONS_HIDDEN_LAYER1 = 3
N_NEURONS_HIDDEN_LAYER2 = 2
N_NEURONS_ENTRIES = 2
N_NEURONS_OUTPUT = 1

##### train case #####
train_set_X_xor = np.array([
    np.reshape([0, 0], (1, 2)),
    # np.reshape([0, 1], (1, 2)),
    np.reshape([1, 0], (1, 2)),
    np.reshape([1, 1], (1, 2)),
])

train_set_Y_xor = np.reshape([
    1,
    # 0,
    0,
    1,
], (3, 1))

##### test case #####
test_set_X_xor = np.array([
    np.reshape([0, 0], (1, 2)),
    np.reshape([0, 1], (1, 2)),
    np.reshape([1, 0], (1, 2)),
    np.reshape([1, 1], (1, 2)),
])

test_set_Y_xor = np.reshape([
    1,
    0,
    0,
    1,
], (4, 1))

##### hyperparameters (W, b) #####
# bias from entries is 0
# b0 = 0

# first layer weights
# [first entry, second entry]
w1 = np.array([
    [np.random.uniform(-1, 1.5), np.random.uniform(-0.9, 1), np.random.uniform(-2.2, 2.5)], # e1
    [np.random.uniform(-0.5, 1), np.random.uniform(0.3, -0.6), np.random.uniform(0.1, 1.4)], # e0
    # we have 6 conexiones, so we get 3 neurons
])

b1 = np.reshape([
    np.random.uniform(-1.5, -0.5), # np.random.uniform(0, 1),
    np.random.uniform(-0.8, 1.5),
    np.random.uniform(-1, 1),
], (1, 3))

print(b1)

# second layer
w2 = np.array([
    # from the 3 neurons of layer 1
    [np.random.uniform(-1, -0.5), np.random.uniform(-1.5, 1.5)], # e^1_1
    [np.random.uniform(-0.5, -2), np.random.uniform(-3, 3.5)], # e^1_2
    [np.random.uniform(-1, 1), np.random.uniform(-1, 1)], # e^1_3
])

b2 = np.reshape([
    np.random.uniform(-1, 1),
    np.random.uniform(-2, 2),
], (1, 2))

# output layer
w3 = np.reshape([
    np.random.uniform(-1, 1), # np.random.uniform(0, 1), # first neuron from hidden layer
    np.random.uniform(-0.5, 0.5), # from second neuron from hidden layer
], (2, 1))

b3 = np.reshape(np.random.uniform(-1, 1), (1, 1))

In [ ]:
w = [w1, w2, w3]
b = [b1, b2, b3]
print(w)
print(b)
hyper_params_after = model(train_set_X_xor, w, b, train_set_Y_xor, epochs=800, plt_all_progress=False,
                           n_layers=N_LAYERS, n_neurons_hidden_l=[N_NEURONS_HIDDEN_LAYER1, N_NEURONS_HIDDEN_LAYER2],
                           learning_rate=.85)

print("-------------------------------------------------------------------")
w_next = [[], [], []]
b_next = [[], [], []]
for i in range(N_LAYERS):
    w_next[N_LAYERS-i-1] = hyper_params_after[i][f"w{N_LAYERS-i}"]
    b_next[N_LAYERS-i-1] = hyper_params_after[i][f"b{N_LAYERS-i}"]

print(w_next)
print(b_next)

In [ ]:
predictions = []
for test in test_set_X_xor:
    predictions.append(predict(test, w_next, b_next, n_layers=3))

for test in range(len(test_set_X_xor)):
    print(f'{test_set_X_xor[test].squeeze()[0]} XOR {test_set_X_xor[test].squeeze()[1]} = {predictions[test]}')

acc = accuracy(predictions, test_set_Y_xor)

print(f"Accuracy: {acc}% from {len(predictions)} tests")